In [ ]:
import sys
!python -m pip install sentinelhub

In [ ]:
#code for autodownloading the sentinel2 images from sentinelhub 
import datetime
import os
import getpass

import matplotlib.pyplot as plt
import numpy as np

#importing the neccesary variables from sentinelhub

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)



# Only run this cell if you have not created a configuration.

config = SHConfig()
config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
#config.sh_client_id = 'a.musunuri@titan4.it'
#config.sh_client_secret = 'Aneesha@7396'



#config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
#config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.sh_base_url = "https://dataspace.copernicus.eu/"
# config.save("cdse")
betsiboka_coords_wgs84 = (46.16, -16.15, 46.51, -15.58)
resolution = 60
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")

evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""


request_true_color = SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C,  # Use the predefined data collection
            time_interval=("2020-06-12", "2020-06-13"),
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox=betsiboka_bbox,
    size=betsiboka_size,
    config=config,
)

true_color_imgs = request_true_color.get_data(save_data=True)
new_folder = 'new_output_folder'  # Define the new folder name
os.makedirs(new_folder, exist_ok=True)  # Create the folder if it doesn't exist
print(f"Data downloaded to the folder: {new_folder}")
for folder, _, filenames in os.walk(new_folder):
    for filename in filenames:
        print(os.path.join(folder, filename))

In [ ]:
from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubRequest,
    MimeType,
    bbox_to_dimensions,
    BBox,
    CRS,
)

# Step 1: Set up your configuration
config = SHConfig()
config.sh_client_id = 'sh-f4a2dffd-e52c-4d3b-9e5b-e1a2787df9b6'  # Replace with your Sentinel Hub client ID
config.sh_client_secret = 'hUZYxAyDlH9WEb3GTnLvowHkJLnBkios'  # Replace with your Sentinel Hub client secret
config.sh_base_url = 'https://dataspace.copernicus.eu/'  # Sentinel Hub base URL

# Step 2: Define request parameters
# Define the bounding box (location) in WGS84 format (latitude, longitude)
# Example coordinates: Adjust as needed
bbox = BBox((12.27, 45.52, 12.30, 45.55), crs=CRS.WGS84)  # Example coordinates

# Define the resolution (in meters) and calculate the image size
resolution = 10  # 10 m resolution
size = bbox_to_dimensions(bbox, resolution)

# Define the time interval (only May 1st, 2020)
time_interval = ('2020-05-01', '2020-05-01')

# Define the Evalscript for NDVI
evalscript = """
//VERSION=3

function setup() {
    return {
        input: ["B04", "B08"],
        output: { id: "default", bands: 1, sampleType: "FLOAT32" }
    };
}

function evaluatePixel(sample) {
    const ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04);
    return [ndvi];
}
"""

# Step 3: Create the SentinelHubRequest
request = SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C,
            time_interval=time_interval,
        )
    ],
    responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)],
    bbox=bbox,
    size=size,
    config=config,
)

# Step 4: Execute the request and download the NDVI image
ndvi_image = request.get_data()[0]

# Step 5: Save the downloaded NDVI image to a file
output_file_path = 'ndvi_image.tiff'
with open(output_file_path, 'wb') as f:
    f.write(ndvi_image)

print(f"NDVI image downloaded and saved to: {output_file_path}")
